In [1]:
import os
import pandas as pd
import numpy as np
import itertools
import openmatrix as omx
from dbfread import DBF
import geopandas as gpd
import folium as fm
from branca.colormap import linear
import ipywidgets as widgets
from ipywidgets import interact

c:\Users\cday\Anaconda3\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\cday\AppData\Local\Temp\ipykernel_25688\3194225179.py:7: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are 

## Purpose

**Assessment #2**: District-to-district shares assessment (analyze where trips are going to and where trips are coming from)

## Inputs

In [2]:
# model data paths
taz_pa_path  = r"../../_large-files/WF-TDM-v9x-v920-E2/PA_AllPurp_GRAVITY.omx"
dmed_pa_path = r"_data/E2.A2/DISTMED_PA_Gravity_AllPurp.omx"
dlrg_pa_path = r"_data/E2.A2/DISTLRG_PA_Gravity_AllPurp.omx"

# hh survey data paths
trips_path   = r"../../_large-files/WF-TDM-v9x-v920-E2/hhsurvey_trips_20221121.csv"
taz_path     = r"../../_large-files/WF-TDM-v9x-v920-E2/hhsurvey_taz_20221121.csv"

# district-taz data path
taz_dist_path = r"_data/E2.A2/WFv910_TAZ.dbf"

## Observed Data

Read in the observed data in production/attraction format. Observed data comes from the 2012 household travel survey. 

In [3]:
# read in hh travel survey
hh_trips = pd.read_csv(trips_path, encoding="ISO-8859-1").reset_index(drop=True)
hh_trips = hh_trips[['tripID','weight',' p_CoTAZID_v30 ',' a_CoTAZID_v30 ','PURP7_t']]

# read in taz shapefile
hh_taz = pd.read_csv(taz_path).reset_index(drop=True)[['SA_TAZID','CO_TAZID','SUBAREAID']]
hh_taz = hh_taz[hh_taz['SUBAREAID']==1]
hh_taz = hh_taz.rename(columns={"SA_TAZID":"TAZID"}).drop(columns="SUBAREAID")

# convert tazids to subarea tazes
hh_trips = hh_trips.merge(hh_taz,left_on=" p_CoTAZID_v30 ",right_on="CO_TAZID",how="left").rename(columns={"TAZID":"p_TAZID"}).drop(columns="CO_TAZID")
hh_trips = hh_trips.merge(hh_taz,left_on=" a_CoTAZID_v30 ",right_on="CO_TAZID",how="left").rename(columns={"TAZID":"a_TAZID"}).drop(columns="CO_TAZID")

# fill in na
hh_trips_s1 = hh_trips[(hh_trips["p_TAZID"].notna()) | (hh_trips["a_TAZID"].notna())]
hh_trips_s1

C:\Users\cday\AppData\Local\Temp\ipykernel_25688\3174631756.py:2: DtypeWarning: Columns (13,46) have mixed types. Specify dtype option on import or set low_memory=False.
  hh_trips = pd.read_csv(trips_path, encoding="ISO-8859-1").reset_index(drop=True)


,tripID,weight,p_CoTAZID_v30,a_CoTAZID_v30,PURP7_t,p_TAZID,a_TAZID
191,3,21.999216,50314,110123,NHBW,NaN,704.0
192,4,21.999216,110141,110123,NHBW,722.0,704.0
193,5,21.999216,110141,50314,NHBW,722.0,NaN
245,1,24.510474,50168,350060,HBW,NaN,965.0
260,2,52.298061,23052,350426,NHBW,NaN,1331.0
...,...,...,...,...,...,...,...
99146,2,90.304727,351205,350987,HBW,2110.0,1892.0
99147,2,111.746606,350734,350738,HBW,1639.0,1643.0
99148,2,308.058765,350433,350531,HBW,1338.0,1436.0
99149,2,308.058765,350433,350531,HBW,1338.0,1436.0


In [4]:
# summarize trips regardless of purpose
grouped_df = hh_trips_s1.groupby(['p_TAZID', 'a_TAZID'], as_index=False)['weight'].sum()
grouped_df = grouped_df.rename(columns={"weight":"trips_obs"})

taz_ids = range(1, 3630)
all_combos = pd.DataFrame(itertools.product(taz_ids, taz_ids), columns=['p_TAZID', 'a_TAZID'])
final_df_tot = all_combos.merge(grouped_df, on=['p_TAZID', 'a_TAZID'], how='left').fillna({'trips_obs': 0})

final_df_tot['Purp'] = 'TOT'

# summarize trips for hbw purpose
hh_trips_hbw = hh_trips_s1[hh_trips_s1['PURP7_t']=='HBW']
grouped_df_hbw = hh_trips_hbw.groupby(['p_TAZID', 'a_TAZID'], as_index=False)['weight'].sum()
grouped_df_hbw = grouped_df_hbw.rename(columns={"weight":"trips_obs"})

final_df_hbw = all_combos.merge(grouped_df_hbw, on=['p_TAZID', 'a_TAZID'], how='left').fillna({'trips_obs': 0})

final_df_hbw['Purp'] = 'HBW'

# concat total and hbw purposes into one long table
final_obs = pd.concat([final_df_tot,final_df_hbw]).reset_index().drop(columns="index")
final_obs

,p_TAZID,a_TAZID,trips_obs,Purp
0,1,1,0.0,TOT
1,1,2,0.0,TOT
2,1,3,0.0,TOT
3,1,4,0.0,TOT
4,1,5,0.0,TOT
...,...,...,...,...
26339277,3629,3625,0.0,HBW
26339278,3629,3626,0.0,HBW
26339279,3629,3627,0.0,HBW
26339280,3629,3628,0.0,HBW


## Model Data

Read in model data omx files and convert to similar long format link the observed. 

In [9]:
# read in omx file
omxFile = omx.open_file(taz_pa_path)

# summarize trips regardless of purpose 
tot = pd.DataFrame(omxFile['TOT'])
tot_flat = tot.reset_index().melt(id_vars='index', var_name='a_TAZID', value_name='trips_mod')
tot_flat.rename(columns={'index': 'p_TAZID'}, inplace=True)
tot_flat['Purp'] = 'TOT'

# summarize trips for hbw purpose
hbw = pd.DataFrame(omxFile['HBW'])
hbw_flat = hbw.reset_index().melt(id_vars='index', var_name='a_TAZID', value_name='trips_mod')
hbw_flat.rename(columns={'index': 'p_TAZID'}, inplace=True)
hbw_flat['Purp'] = 'HBW'

# concat total and hbw purposes into one long table
final_mod = pd.concat([tot_flat,hbw_flat])
final_mod

# fix index since cube starts at 1 and pythons tarts at 0
final_mod['p_TAZID'] = final_mod['p_TAZID'].astype(int) + 1
final_mod['a_TAZID'] = final_mod['a_TAZID'].astype(int) + 1

## Compare Observed and Model Trip Data

In this section we compare the observed and model trip data both with a map and some important tables. We do it at the large district level, but the code is written so we can do lower levels with relatively little changes if needed. This specific section before the map and table section shows data at multip.le geographic levels

In [10]:
# merge observed and model data
final_pa = final_obs.merge(final_mod, on=['p_TAZID','a_TAZID','Purp'], how='left').fillna(0)
final_taz = final_pa.rename(columns={"p_TAZID":"p",'a_TAZID':'a'})
final_taz['geo'] = 'TAZID'

In [11]:
# read in taz shapefile
taz_dist = pd.DataFrame(iter(DBF(taz_dist_path, load=True)))[['TAZID','DISTLRG','DISTMED','DISTSML']]

# merge on p to get districts
final_pad = final_pa.merge(taz_dist,left_on="p_TAZID",right_on='TAZID', how='left')
final_pad = final_pad.rename(columns={'DISTLRG':'p_DISTLRG','DISTMED':'p_DISTMED','DISTSML':'p_DISTSML'})

# merge on a to get districts
final_pad = final_pad.merge(taz_dist,left_on="a_TAZID",right_on='TAZID', how='left')
final_pad = final_pad.rename(columns={'DISTLRG':'a_DISTLRG','DISTMED':'a_DISTMED','DISTSML':'a_DISTSML'})

# drop columns
final_pad = final_pad.drop(columns={'TAZID_x','TAZID_y'})

In [12]:
# reformate lrg districts
final_dlrg = final_pad[['p_DISTLRG','a_DISTLRG','trips_obs','trips_mod', 'Purp']]
final_dlrg = final_dlrg.groupby(['p_DISTLRG', 'a_DISTLRG','Purp'], as_index=False).agg({"trips_obs": "sum","trips_mod": "sum"})
final_dlrg['geo'] = 'DISTLRG'
final_dlrg = final_dlrg.rename(columns={'p_DISTLRG':'p','a_DISTLRG':'a'})

# reformate med districts
final_dmed = final_pad[['p_DISTMED','a_DISTMED','trips_obs','trips_mod', 'Purp']]
final_dmed = final_dmed.groupby(['p_DISTMED', 'a_DISTMED','Purp'], as_index=False).agg({"trips_obs": "sum","trips_mod": "sum"})
final_dmed['geo'] = 'DISTMED'
final_dmed = final_dmed.rename(columns={'p_DISTMED':'p','a_DISTMED':'a'})

# reformate sml districts
final_dsml = final_pad[['p_DISTSML','a_DISTSML','trips_obs','trips_mod', 'Purp']]
final_dsml = final_dsml.groupby(['p_DISTSML', 'a_DISTSML','Purp'], as_index=False).agg({"trips_obs": "sum","trips_mod": "sum"})
final_dsml['geo'] = 'DISTSML'
final_dsml = final_dsml.rename(columns={'p_DISTSML':'p','a_DISTSML':'a'})

In [13]:
# combine all into one table
final_df = pd.concat([final_taz,final_dlrg,final_dmed,final_dsml])
final_df = final_df[['geo','Purp','p','a','trips_mod','trips_obs']]
final_df

,geo,Purp,p,a,trips_mod,trips_obs
0,TAZID,TOT,1.0,1.0,0.0,0.0
1,TAZID,TOT,1.0,2.0,0.0,0.0
2,TAZID,TOT,1.0,3.0,0.0,0.0
3,TAZID,TOT,1.0,4.0,0.0,0.0
4,TAZID,TOT,1.0,5.0,0.0,0.0
...,...,...,...,...,...,...
33795,DISTSML,TOT,130.0,128.0,0.0,0.0
33796,DISTSML,HBW,130.0,129.0,0.0,0.0
33797,DISTSML,TOT,130.0,129.0,0.0,0.0
33798,DISTSML,HBW,130.0,130.0,0.0,0.0


### Map Comparison (Large Districts)

View comparisons on a map at the large district level

In [15]:
# read in large district file
dist_lrg = gpd.read_file(r"D:\GitHub\WF-TDM-v9x\1_Inputs\1_TAZ\Districts\Dist_Large.shp")[['DISTLRG','DLRG_NAME','geometry']]
dist_lrg['DISTLRG'] = dist_lrg['DISTLRG'].astype(float)

In [16]:
# function to return colormap
def get_colormap(maxVal):
    return linear.YlGnBu_09.scale(0, maxVal)

# function to return colormap axes
def get_colormap_axes(maxVal):
    return linear.YlGnBu_09.scale(0.0, maxVal)

# function to get map
def get_map(lat=40.2338, long=-111.6585, zoom_start=9):
    f = fm.Figure(width=1000, height=1000)
    return fm.Map(location=[lat, long], zoom_start=zoom_start, tiles='cartodbpositron').add_to(f)

In [17]:
def update_map(geo, purp, pa, pa_name, trip, zoom): #, maxVal):
    map = get_map()
    
    # filter to correct geo values
    if geo=='DISTLRG':
        geo_name = 'DLRG_NAME'
        geo_shp  = dist_lrg
    
    # filter df
    df = final_df.loc[(final_df['geo'] == geo) & (final_df['Purp'] == purp)]
    gdf = gpd.GeoDataFrame(df.merge(geo_shp, left_on=pa, right_on=geo))
    
    # determine correct geo to be analysis area
    pa_num = float(gdf.loc[gdf[geo_name] == pa_name, geo].values[0])
    if pa == 'p': 
        gdf = gdf.loc[gdf['a'] == pa_num]
    elif pa == 'a': 
        gdf = gdf.loc[gdf['p'] == pa_num]
    
    # get maximum trip value
    maxVal = int(gdf[trip].max())
    
    # debug if gdf is empty
    if gdf.empty:
        print("No data available for the selected filters.")
        print(gdf)
        return map

    # Choose colormap based on zoom option
    colormap = get_colormap(int(maxVal)) if zoom == 'AutoFit' else get_colormap_axes(float(maxVal))
    
    # style function for color
    style_function = lambda x: {
        'fillColor': colormap(x['properties'][trip]),
        'color': 'black',
        'weight': 0.25,
        'fillOpacity': 0.7
    }
    
    # create map
    fm.GeoJson(
        gdf,
        style_function=style_function,
        tooltip=fm.GeoJsonTooltip(
            fields=[geo_name, trip],
            aliases=["geo_name: ", 'Number of Trips: '],
            localize=True
        )
    ).add_to(map)
    
    colormap.add_to(map)
    
    return map


In [18]:
# Widgets for interaction
lstGeo = ['DISTLRG']#, 'DISTMED', 'DISTSML']
lstPurp = ['TOT', 'HBW']
lstPA = ['p', 'a']
lstPAName = dist_lrg['DLRG_NAME'].tolist()
lstTrip = ['trips_obs', 'trips_mod']
lstZoom = ['AutoFit', 'Selection']
#mMax = widgets.Text(value='1500', description='Max')

# Ensure update_map is defined before using interact
interact(update_map, geo=lstGeo, purp=lstPurp, pa=lstPA, pa_name=lstPAName, trip=lstTrip, zoom=lstZoom)#, maxVal=mMax)

interactive(children=(Dropdown(description='geo', options=('DISTLRG',), value='DISTLRG'), Dropdown(description…

<function __main__.update_map(geo, purp, pa, pa_name, trip, zoom)>

### Table Comparison (Large District & County)

View district to district flows at the large district and county level. We really zoom into where are trips that 

In [19]:
# filter to large districts
df_dist = final_df.loc[(final_df['geo'] == 'DISTLRG')]
df_dist = df_dist.merge(dist_lrg, left_on='a', right_on='DISTLRG')

# calcualte totals of trips within the same (geo,purp,p) combo 
df_dist['total_trips_mod'] = df_dist.groupby(['Purp','p'])['trips_mod'].transform('sum')
df_dist['total_trips_obs'] = df_dist.groupby(['Purp','p'])['trips_obs'].transform('sum')

# calculate percentage of trips attracted to each 'a'
df_dist['trips_mod_perc'] = (df_dist['trips_mod'] / df_dist['total_trips_mod']) * 100
df_dist['trips_obs_perc'] = (df_dist['trips_obs'] / df_dist['total_trips_obs']) * 100

# select only needed columns
df_dist = df_dist[['geo','Purp','p','a','trips_mod','trips_obs','trips_mod_perc','trips_obs_perc','DLRG_NAME']]

# table showing of all districts, how many are being attracted to that district from Utah County Central (Provo/Orem Region)
df_dist_a_provo = df_dist[df_dist['p']==22].fillna(0)
df_dist_a_provo_hbw = df_dist_a_provo[df_dist_a_provo['Purp']=='HBW']
df_dist_a_provo_tot = df_dist_a_provo[df_dist_a_provo['Purp']=='TOT']

In [20]:
df_dist_a_provo_tot

,geo,Purp,p,a,trips_mod,trips_obs,trips_mod_perc,trips_obs_perc,DLRG_NAME
1093,DISTLRG,TOT,22.0,1.0,0.04,0.000000,0.000006,0.000000,Box Elder County - WFRC
1095,DISTLRG,TOT,22.0,2.0,0.00,0.000000,0.000000,0.000000,Box Elder County - not WFRC
1097,DISTLRG,TOT,22.0,3.0,0.00,0.000000,0.000000,0.000000,Weber County - Great Slat Lake
1099,DISTLRG,TOT,22.0,4.0,114.88,152.759292,0.016290,0.041164,Weber County - North and West
1101,DISTLRG,TOT,22.0,5.0,497.85,271.735912,0.070596,0.073225,Weber County - Ogden Area
1103,DISTLRG,TOT,22.0,6.0,0.00,0.000000,0.000000,0.000000,Weber County - East Mountains
1105,DISTLRG,TOT,22.0,7.0,0.00,0.000000,0.000000,0.000000,Davis County - Great Salt Lake
1107,DISTLRG,TOT,22.0,8.0,832.07,0.000000,0.117989,0.000000,Davis County - North
1109,DISTLRG,TOT,22.0,9.0,1260.76,315.412421,0.178777,0.084995,Davis County - South
1111,DISTLRG,TOT,22.0,10.0,0.00,0.000000,0.000000,0.000000,Davis County - East Mountains


In [21]:
df_dist_a_provo_hbw

,geo,Purp,p,a,trips_mod,trips_obs,trips_mod_perc,trips_obs_perc,DLRG_NAME
1092,DISTLRG,HBW,22.0,1.0,0.00,0.000000,0.000000,0.000000,Box Elder County - WFRC
1094,DISTLRG,HBW,22.0,2.0,0.00,0.000000,0.000000,0.000000,Box Elder County - not WFRC
1096,DISTLRG,HBW,22.0,3.0,0.00,0.000000,0.000000,0.000000,Weber County - Great Slat Lake
1098,DISTLRG,HBW,22.0,4.0,5.43,0.000000,0.006028,0.000000,Weber County - North and West
1100,DISTLRG,HBW,22.0,5.0,12.68,0.000000,0.014076,0.000000,Weber County - Ogden Area
1102,DISTLRG,HBW,22.0,6.0,0.00,0.000000,0.000000,0.000000,Weber County - East Mountains
1104,DISTLRG,HBW,22.0,7.0,0.00,0.000000,0.000000,0.000000,Davis County - Great Salt Lake
1106,DISTLRG,HBW,22.0,8.0,126.89,0.000000,0.140861,0.000000,Davis County - North
1108,DISTLRG,HBW,22.0,9.0,390.38,180.197798,0.433361,0.302587,Davis County - South
1110,DISTLRG,HBW,22.0,10.0,0.00,0.000000,0.000000,0.000000,Davis County - East Mountains


In [22]:
# Sample DataFrame
df_co_a_provo = df_dist_a_provo.copy()
df_co_a_provo['County'] = np.select(
    [
        df_co_a_provo['DLRG_NAME'].str.contains('Box Elder', na=False),
        df_co_a_provo['DLRG_NAME'].str.contains('Utah', na=False),
        df_co_a_provo['DLRG_NAME'].str.contains('Davis', na=False),
        df_co_a_provo['DLRG_NAME'].str.contains('Weber', na=False),
        df_co_a_provo['DLRG_NAME'].str.contains('Salt Lake', na=False),
    ],
    ['Box Elder', 'Utah', 'Davis', 'Weber', 'Salt Lake'],
    default='Unknown'  # Default value if none match
)


df_co_a_provo_hbw = df_co_a_provo[df_co_a_provo['Purp']=='HBW']
df_co_a_provo_tot = df_co_a_provo[df_co_a_provo['Purp']=='TOT']

df_co_a_provo_hbw['trips_mod_perc_co'] = df_co_a_provo_hbw.groupby(['County', 'Purp'])['trips_mod_perc'].transform('sum')
df_co_a_provo_hbw['trips_obs_perc_co'] = df_co_a_provo_hbw.groupby(['County', 'Purp'])['trips_obs_perc'].transform('sum')
df_co_a_provo_tot['trips_mod_perc_co'] = df_co_a_provo_tot.groupby(['County', 'Purp'])['trips_mod_perc'].transform('sum')
df_co_a_provo_tot['trips_obs_perc_co'] = df_co_a_provo_tot.groupby(['County', 'Purp'])['trips_obs_perc'].transform('sum')

C:\Users\cday\AppData\Local\Temp\ipykernel_25688\2656595202.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_co_a_provo_hbw['trips_mod_perc_co'] = df_co_a_provo_hbw.groupby(['County', 'Purp'])['trips_mod_perc'].transform('sum')
C:\Users\cday\AppData\Local\Temp\ipykernel_25688\2656595202.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_co_a_provo_hbw['trips_obs_perc_co'] = df_co_a_provo_hbw.groupby(['County', 'Purp'])['trips_obs_perc'].transform('sum')
C:\Users\cday\AppData\Local\Temp\ipykernel

In [26]:
df_co_a_provo_tot

,geo,Purp,p,a,trips_mod,trips_obs,trips_mod_perc,trips_obs_perc,DLRG_NAME,County,trips_mod_perc_co,trips_obs_perc_co
1093,DISTLRG,TOT,22.0,1.0,0.04,0.000000,0.000006,0.000000,Box Elder County - WFRC,Box Elder,0.000006,0.000000
1095,DISTLRG,TOT,22.0,2.0,0.00,0.000000,0.000000,0.000000,Box Elder County - not WFRC,Box Elder,0.000006,0.000000
1097,DISTLRG,TOT,22.0,3.0,0.00,0.000000,0.000000,0.000000,Weber County - Great Slat Lake,Weber,0.086886,0.114390
1099,DISTLRG,TOT,22.0,4.0,114.88,152.759292,0.016290,0.041164,Weber County - North and West,Weber,0.086886,0.114390
1101,DISTLRG,TOT,22.0,5.0,497.85,271.735912,0.070596,0.073225,Weber County - Ogden Area,Weber,0.086886,0.114390
1103,DISTLRG,TOT,22.0,6.0,0.00,0.000000,0.000000,0.000000,Weber County - East Mountains,Weber,0.086886,0.114390
1105,DISTLRG,TOT,22.0,7.0,0.00,0.000000,0.000000,0.000000,Davis County - Great Salt Lake,Davis,0.296766,0.084995
1107,DISTLRG,TOT,22.0,8.0,832.07,0.000000,0.117989,0.000000,Davis County - North,Davis,0.296766,0.084995
1109,DISTLRG,TOT,22.0,9.0,1260.76,315.412421,0.178777,0.084995,Davis County - South,Davis,0.296766,0.084995
1111,DISTLRG,TOT,22.0,10.0,0.00,0.000000,0.000000,0.000000,Davis County - East Mountains,Davis,0.296766,0.084995


In [27]:
df_co_a_provo_hbw

,geo,Purp,p,a,trips_mod,trips_obs,trips_mod_perc,trips_obs_perc,DLRG_NAME,County,trips_mod_perc_co,trips_obs_perc_co
1092,DISTLRG,HBW,22.0,1.0,0.00,0.000000,0.000000,0.000000,Box Elder County - WFRC,Box Elder,0.000000,0.000000
1094,DISTLRG,HBW,22.0,2.0,0.00,0.000000,0.000000,0.000000,Box Elder County - not WFRC,Box Elder,0.000000,0.000000
1096,DISTLRG,HBW,22.0,3.0,0.00,0.000000,0.000000,0.000000,Weber County - Great Slat Lake,Weber,0.020104,0.000000
1098,DISTLRG,HBW,22.0,4.0,5.43,0.000000,0.006028,0.000000,Weber County - North and West,Weber,0.020104,0.000000
1100,DISTLRG,HBW,22.0,5.0,12.68,0.000000,0.014076,0.000000,Weber County - Ogden Area,Weber,0.020104,0.000000
1102,DISTLRG,HBW,22.0,6.0,0.00,0.000000,0.000000,0.000000,Weber County - East Mountains,Weber,0.020104,0.000000
1104,DISTLRG,HBW,22.0,7.0,0.00,0.000000,0.000000,0.000000,Davis County - Great Salt Lake,Davis,0.574222,0.302587
1106,DISTLRG,HBW,22.0,8.0,126.89,0.000000,0.140861,0.000000,Davis County - North,Davis,0.574222,0.302587
1108,DISTLRG,HBW,22.0,9.0,390.38,180.197798,0.433361,0.302587,Davis County - South,Davis,0.574222,0.302587
1110,DISTLRG,HBW,22.0,10.0,0.00,0.000000,0.000000,0.000000,Davis County - East Mountains,Davis,0.574222,0.302587


## Conclusions

- the majority of the trips produced in the *Utah County - Utah Valley Central* district are attracted to a location within utah county (~96% of trips)
- the model is underpredicting trips produced in *Utah County - Utah Valley Central* district and attracted to districts within Salt Lake County by ~1%
    - these trips are for non home-based work trips (HBW is predicting this flow accurately within 0.03%)
- this could be playing a slight role in the under-represented boardings at Provo Station
